In [3]:
import numpy as np
import platform
import tempfile
import os
import math
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from skimage import io, color
from scipy.fftpack import dct

In [4]:
im=io.imread('pelican99.jpg')
im = color.rgb2ycbcr(im)


In [ ]:
def im_luminance(im):
    if len(im.shape) == 3:
        R, G, B = im[:,:,0], im[:,:,1], im[:,:,2]
        luminance = 0.299*R + 0.587*G + 0.114*B
    else:
        luminance = im
    return luminance

In [ ]:
def voteMAP(image, X, Y):
    cos_t = [[0 for _ in range(8)] for _ in range(8)] # Table utile pour la DCT
    for k in range(8):
        for l in range(8):
            cos_t[k][l] = math.cos((2.0 * k + 1.0) * l * math.pi / 16.0)

    votes = [-1] * (X * Y) # Table des votes

    # DCT par blocs de 8x8 pixels
    for x in range(X - 7):
        for y in range(Y - 7):
            z = 0 # Nombre de zeros dans le bloc
            const_along_x = True
            const_along_y = True
            while (const_along_x or const_along_y): #On verifie que le bloc n'est pas constant
                for xx in range(8):
                    for yy in range(8):
                        if image[x + xx + (y + yy) * X] != image[x + 0 + (y + yy) * X]: #On verifie que le bloc n'est pas constant selon x
                            const_along_x = False
                        if image[x + xx + (y + yy) * X] != image[x + xx + (y + 0) * X]: #On verifie que le bloc n'est pas constant selon y
                            const_along_y = False

            for i in range(8): # On calcule la DCT du bloc
                for j in range(8):
                    if i > 0 or j > 0:  # Le coefficient (0,0) est constant donc on le prend pas en compte
                        dct_ij = 0.0
                        for xx in range(8):
                            for yy in range(8):
                                dct_ij += image[x + xx + (y + yy) * X] * cos_t[xx][i] * cos_t[yy][j]
                        dct_ij *= 0.25 * (1.0 / math.sqrt(2.0) if i == 0 else 1.0) * (1.0 / math.sqrt(2.0) if j == 0 else 1.0)
                        if abs(dct_ij) < 0.5:
                            z += 1
                            
            for xx in range(x, x + 8): # On vote pour le bloc
                for yy in range(y, y + 8):
                    if z > 0 and z > votes[xx + yy * X]:
                        votes[xx + yy * X] = -1 if const_along_x or const_along_y else (x % 8) + (y % 8) * 8

    # On elimine les bordures de l'image
    for xx in range(X):
        for yy in range(7):
            votes[xx + yy * X] = -1
        for yy in range(Y - 7, Y):
            votes[xx + yy * X] = -1
    for yy in range(Y):
        for xx in range(7):
            votes[xx + yy * X] = -1
        for xx in range(X - 7, X):
            votes[xx + yy * X] = -1
    
    return votes


In [5]:
def votesforeachpixel(im) :
    shape = im.shape
    X = shape[0]
    Y = shape[1]  
    zeros = np.zeros((X,Y))
    votes = np.zeros((X,Y))
    for x in range(X-7) :
        for y in range(Y-7) :
            block = im[x:x+8,y:y+8,0]
            constantx = False
            constanty = False

            #on regarde si le block est constant selon x ou y
            for xx in range(8) :
                for yy in range(8) :
                    if block[xx,yy] == block [xx,0] :
                        constanty = True

                    if block[xx,yy] == block [0,yy] :
                        constantx = True

            dctblock = dct(dct(block, axis=0, norm='ortho'), axis=1, norm='ortho')
            nbzeros = 0

            for i in range(8) :
                for j in range(8) :
                    if abs(dctblock[i,j])<0.5 :
                        nbzeros+=1
            for xx in range(8) :
                for yy in range(8) :
                    if nbzeros>zeros[x+xx,y+yy] :
                        zeros[x+xx,y+yy] = nbzeros
                        if not (constantx or constanty) :
                            votes[x+xx,y+yy]= (x%8,y%8)
    return votes

In [6]:
def detectgrid (im) :
    shape = im.shape
    X = shape[0]
    Y = shape[1]  
    votes = votesforeachpixel(im)
    gridvotes = np.zeros((8,8))
    maxvote =0
    mostvotedgrid = 0
    for x in range(X) :
        for y in range(Y) :
            if votes[x,y] != 0 :
                gridvotes[votes[x,y]]+=1
                if gridvotes[votes[x,y]]> maxvote :
                    maxvote = gridvotes[votes[x,y]]
                    mostvotedgrid = votes[x,y]
    # il reste à coder le calcul du nfa
    return mostvotedgrid, maxvote

In [7]:
print(detectgrid(im))

(0, 0)


In [ ]:
def zero(input, input_jpeg, luminance, luminance_jpeg, votes, votes_jpeg, lnfa_grids, foreign_regions, foreign_regions_n, missing_regions, missing_regions_n, mask_f, mask_f_reg, mask_m, mask_m_reg, X, Y, C, C_jpeg):
    main_grid = -1
    rgb2luminance(input, luminance, X, Y, C)
    compute_grid_votes_per_pixel(luminance, votes, X, Y)
    main_grid = detect_global_grids(votes, lnfa_grids, X, Y)
    foreign_regions_n[0] = detect_forgeries(votes, mask_f, mask_f_reg, foreign_regions, X, Y, main_grid, 63)
    if main_grid > -1 and input_jpeg is not None:
        rgb2luminance(input_jpeg, luminance_jpeg, X, Y, C_jpeg)
        compute_grid_votes_per_pixel(luminance_jpeg, votes_jpeg, X, Y)
        for x in range(X):
            for y in range(Y):
                if votes[x+y*X] == main_grid:
                    votes_jpeg[x+y*X] = -1
        missing_regions_n[0] = detect_forgeries(votes_jpeg, mask_m, mask_m_reg, missing_regions, X, Y, -1, 0)

    return main_grid